# Import Libraries

In [ ]:
import sys
import mosek
import numpy as np
import pandas as pd

# Load Dataset

In [ ]:
df_machines = pd.read_csv('../Datasets/Real-life Dataset/Machines_250.csv')
df_applications = pd.read_csv('../Datasets/Real-life Dataset/Applications_200.csv')
df_anti_affinity_matrix = pd.read_csv('../Datasets/Real-life Dataset/Anti_Affinity_Matrix_200_250.csv')
df_affinity_matrix = pd.read_csv('../Datasets/Real-life Dataset/Affinity_Matrix_200_250.csv')

# df_machines = pd.read_csv('../Datasets/Synthetic Dataset/Machines.csv')
# df_applications = pd.read_csv('../Datasets/Synthetic Dataset/Applications.csv')
# df_anti_affinity_matrix = pd.read_csv('../Datasets/Synthetic Dataset/Anti_Affinity_Matrix.csv')
# df_affinity_matrix = pd.read_csv('../Datasets/Synthetic Dataset/Affinity_Matrix.csv')

# df_anti_affinity_matrix = pd.read_csv('../Datasets/Anti-Affinity Matrices/Anti_Affinity_Matrix_10.csv')
# df_anti_affinity_matrix = pd.read_csv('../Datasets/Anti-Affinity Matrices/Anti_Affinity_Matrix_20.csv')
# df_anti_affinity_matrix = pd.read_csv('../Datasets/Anti-Affinity Matrices/Anti_Affinity_Matrix_30.csv')
# df_anti_affinity_matrix = pd.read_csv('../Datasets/Anti-Affinity Matrices/Anti_Affinity_Matrix_40.csv')
# df_anti_affinity_matrix = pd.read_csv('../Datasets/Anti-Affinity Matrices/Anti_Affinity_Matrix_50.csv')

In [ ]:
df_machines = df_machines.loc[:, ~df_machines.columns.str.contains('^Unnamed')]
df_applications = df_applications.loc[:, ~df_applications.columns.str.contains('^Unnamed')]
df_anti_affinity_matrix = df_anti_affinity_matrix.loc[:, ~df_anti_affinity_matrix.columns.str.contains('^Unnamed')]
df_affinity_matrix = df_affinity_matrix.loc[:, ~df_affinity_matrix.columns.str.contains('^Unnamed')]

In [ ]:
df_anti_affinity_matrix.columns = pd.to_numeric(df_anti_affinity_matrix.columns, errors='coerce')
df_affinity_matrix.columns = pd.to_numeric(df_affinity_matrix.columns, errors='coerce')

# Constants Initialization

In [ ]:
ALPHA = 4
GLOBAL_ANS = []
GLOBAL_ANS_QUAD = []
inf = 0.0

# affinity weights
WEIGHTS = {
    'CPU' : 0.4,
    'IO' : 0.2,
    'NW' : 0.2,
    'Memory' : 0.2
}

# Specify number of application requests and number of machines in the cluster
NUM_machines = 250
NUM_applications = 200

In [ ]:
df_user_affinity_matrix = df_affinity_matrix.copy()

# Calculate System Affinity Matrix

In [ ]:
affinity = np.zeros((df_applications.shape[0], df_machines.shape[0]))
for i, application in df_applications.iterrows():
    for j, machine in df_machines.iterrows():
        for col in ['CPU','IO','NW','Memory']:
            if (application[col] > machine[col]):
                affinity[i][j] = 0
                break;
            else:
                affinity[i][j] +=  WEIGHTS[col] * (machine[col] - application[col]) / machine[col]
df_affinity_matrix = (affinity + df_affinity_matrix) / 2

In [ ]:
df_machines = df_machines.head(NUM_machines)
df_applications = df_applications.head(NUM_applications)

# GCCS

In [ ]:
def get_c():
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  C=[]
  for app in range(num_applications):
    for mac in range(num_machines):
      part1 = (df_machines.loc[mac,'P_max']-df_machines.loc[mac,'P_idle'])*df_applications.loc[app,'CPU']*df_applications.loc[app,'Instances']
      
      part1 = part1/df_machines.loc[mac,'CPU']
      part1 = part1 - (ALPHA*(df_affinity_matrix.loc[app,mac])*df_applications.loc[app,'Instances'])
      
      C.append(part1)
  return C

In [ ]:
def get_A():
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  A=[]
  # anti-affinity constraint
  list1=[]
  for app in range(num_applications):
    for mac in range(num_machines):
      list1.append(df_anti_affinity_matrix.loc[app,mac])
  A.append(list1)
  # all instances must be scheduled
  for app1 in range(num_applications):
    list_temp=[]
    for app in range(num_applications):
      for mac in range(num_machines):
        if(app1==app):
          list_temp.append(1)
        else:
          list_temp.append(0)
    A.append(list_temp)
  # constraint satisfaction
  for col in ['CPU','IO','NW','Memory']:
    for mac1 in range(num_machines):
      list_temp=[]
      for app in range(num_applications):
        for mac in range(num_machines):
          if(mac1==mac):
            list_temp.append(df_applications.loc[app,'Instances']*df_applications.loc[app,col])
            # list_temp.append(df_applications.loc[app,col])
          else:
            list_temp.append(0)
      A.append(list_temp)
  return A

In [ ]:
def get_var_l():
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  num_vars = num_machines*num_applications
  L=[0.0 for i in range(num_vars)]
  return L

In [ ]:
def get_var_u():
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  num_vars = num_machines*num_applications
  U=[+inf for i in range(num_vars)]
  return U

In [ ]:
def get_var_k():
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  num_vars = num_machines*num_applications
  K=[mosek.boundkey.lo for i in range(num_vars)]
  return K

In [ ]:
def get_con_l():
  L=[]
  L.append(0.0)
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  for i in range(num_applications):
    # L.append(df_applications.loc[i,'Instances'])
    L.append(1.0)
  for col in ['CPU','IO','NW','Memory']:
    for mac1 in range(num_machines):
      L.append(-inf)
  return L

In [ ]:
def get_con_u():
  U=[]
  U.append(0.0)
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  for i in range(num_applications):
    U.append(1.0)
    # U.append(df_applications.loc[i,'Instances'])
  for col in ['CPU','IO','NW','Memory']:
    for mac in range(num_machines):
      U.append(df_machines.loc[mac,col])
  return U

In [ ]:
def get_con_k():
  K=[]
  K.append(mosek.boundkey.fx)
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  for i in range(num_applications):
    K.append(mosek.boundkey.fx)
  for col in ['CPU','IO','NW','Memory']:
    for mac in range(num_machines):
      K.append(mosek.boundkey.up)
  return K

In [ ]:
def get_parts(A):
  A_sub=[]
  A_val=[]
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  num_cols = num_machines *num_applications
  num_rows = 1+num_applications +4*num_machines
  for col in range(num_cols):
    sub_temp=[]
    val_temp=[]
    for row in range(num_rows):
      if(A[row][col]!=0):
        sub_temp.append(row)
        val_temp.append(A[row][col])
    A_sub.append(sub_temp)
    A_val.append(val_temp)
  return A_sub,A_val

In [ ]:
#  Define a stream printer to grab output from MOSEK
def streamprinter(text):
    sys.stdout.write(text)
    sys.stdout.flush()

In [ ]:
def main():
  with mosek.Task() as task:
    # Attach a log stream printer to the task
    task.set_Stream(mosek.streamtype.log, streamprinter)
    bkc = get_con_k()
    blc = get_con_l()
    buc = get_con_u()
    bkx = get_var_k()
    blx = get_var_l()
    bux = get_var_u()
    c =get_c()
    A = get_A()
    asub,aval=get_parts(A)
    numvar=len(bkx)
    numcon=len(bkc)
    task.appendcons(numcon)
    task.appendvars(numvar)
    for j in range(numvar):
      # Set the linear term c_j in the objective.
      task.putcj(j, c[j])

      # Set the bounds on variable j
      # blx[j] <= x_j <= bux[j]
      task.putvarbound(j, bkx[j], blx[j], bux[j])

      # Input column j of A
      task.putacol(j,                  # Variable (column) index.
                    asub[j],            # Row index of non-zeros in column j.
                    aval[j])
    for i in range(numcon):
            task.putconbound(i, bkc[i], blc[i], buc[i])
    task.putobjsense(mosek.objsense.minimize)
    task.optimize()
    task.solutionsummary(mosek.streamtype.msg)
    solsta = task.getsolsta(mosek.soltype.bas)
    if (solsta == mosek.solsta.optimal):
            xx = task.getxx(mosek.soltype.bas)
            global GLOBAL_ANS
            GLOBAL_ANS = xx
    elif (solsta == mosek.solsta.dual_infeas_cer or
          solsta == mosek.solsta.prim_infeas_cer):
        print("Primal or dual infeasibility certificate found.\n")
    elif solsta == mosek.solsta.unknown:
        print("Unknown solution status")
    else:
        print("Other solution status")

In [ ]:
try:
    main()
except mosek.MosekException as msg:
    #print "ERROR: %s" % str(code)
    if msg is not None:
        print("\t%s" % msg)
        sys.exit(1)
except:
    import traceback
    traceback.print_exc()
    sys.exit(1)

In [ ]:
# looking at B*
def get_B_star():
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  B=[]
  for app in range(num_applications):
      b_temp=[]
      for mac in range(num_machines):
        b_temp.append(GLOBAL_ANS[app*num_machines + mac]*df_applications.loc[app,'Instances'])
      B.append(b_temp)
  return B

# Utility Functions

In [ ]:
def calculate_utilization(df_allocation_matrix, df_applications, df_machines):
    df_allocation_copy = df_allocation_matrix.copy()
    for i, application in df_applications.iterrows():

        df_allocation_copy.loc[i] = df_allocation_copy.loc[i] * application['CPU']
    return df_allocation_copy.sum() / df_machines['CPU']


In [ ]:
def calculate_total_power_cost(df_allocation_matrix, df_applications, df_machines):
    utilization = calculate_utilization(df_allocation_matrix, df_applications, df_machines)
    power_consumed = df_machines['P_idle'] + (df_machines['P_max'] - df_machines['P_idle']) * (utilization ** 3)
    return power_consumed.sum()

In [ ]:
def calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix):

    # affinity_cost = df_allocation_matrix * df_affinity_matrix
    affinity_cost  = df_allocation_matrix.copy()
    num_machines = df_machines.shape[0]
    num_applications = df_applications.shape[0]
    for app in range(num_applications):
      for mac in range(num_machines):
        affinity_cost.loc[app,mac] = affinity_cost.loc[app,mac]*df_affinity_matrix.loc[app,mac]

    return affinity_cost.sum().sum()

In [ ]:
def calculate_total_system_cost(power_cost, affinity_cost):
    total_cost = power_cost - ALPHA * affinity_cost
    return total_cost

In [ ]:

def calculate_number_of_unused_machines(df_allocation_matrix):
    col_sums = df_allocation_matrix.sum()
    zero_count = (col_sums == 0).sum()
    return zero_count

In [ ]:
def calculate_ASR(df_allocation_matrix):
  df_allocation_copy = df_allocation_matrix.copy()
  # num_instances = df_allocation_copy.sum().sum()
  num_instances = df_applications['Instances'].sum()
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  for app in range(num_applications):
    for mac in range(num_machines):
      df_allocation_copy.loc[app,mac] = df_allocation_copy.loc[app,mac]*df_user_affinity_matrix.loc[app,mac]
  # df_allocation_copy = df_allocation_copy*df_user_affinity_matrix
  num_affine = df_allocation_copy.sum().sum()
  return num_affine/num_instances

In [ ]:
def calculate_AU(df_allocation_matrix):
  utilization = calculate_utilization(df_allocation_matrix, df_applications, df_machines)
  return utilization.mean()

In [ ]:
# print(len(GLOBAL_ANS))
df_allocation_matrix = GLOBAL_ANS_QUAD
B_star = get_B_star()

# Converting to Integer solution

In [ ]:
def find_resources_left(df_allocation_matrix):
    df_remaining = df_machines.copy()
    for m in range(df_machines.shape[0]):
        for a in range(df_applications.shape[0]):
            for col in ['CPU','NW','IO','Memory']:
                df_remaining.loc[m,col] = df_remaining.loc[m,col] - df_allocation_matrix.loc[a,m]*df_applications.loc[a,col]
    return df_remaining

In [ ]:
def find_max(df_allocation_matrix):
  max_r =0
  max_c =0
  max_val=0
  for m in range(df_machines.shape[0]):
        for a in range(df_applications.shape[0]):
          if(df_allocation_matrix.loc[max_r,max_c]<df_allocation_matrix.loc[a,m]):
            max_r=a
            max_c=m
            max_val = df_allocation_matrix.loc[max_r,max_c]
  return max_r,max_c,max_val

In [ ]:
def get_B_final(B_star):
  B_star = pd.DataFrame(B_star)
  B_dash = B_star.copy()
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  for app in range(num_applications):
    for mac in range(num_machines):
      B_dash.loc[app,mac]=int(B_star.loc[app,mac])
  B_star_res = B_star-B_dash
  v_remain = find_resources_left(B_dash)
  Res_instances = df_applications['Instances']-B_dash.sum(axis=1)
  # Res
  while Res_instances.sum() !=0 :
    pos_r,pos_c,val = find_max(B_star_res)
    # print(pos_r,pos_c,val)
    if(val==-1):
      print('hadd hai!!')
      break
    if(Res_instances[pos_r]==0):
      B_star_res.loc[pos_r,pos_c]=-1
      continue
    yes =1
    for col in ['CPU','IO','NW','Memory']:
      if(v_remain.loc[pos_c,col] - df_applications.loc[pos_r,col]<0):
        yes=0
    if(yes==1):
      B_dash.loc[pos_r,pos_c] = B_dash.loc[pos_r,pos_c] +1
      for col in ['CPU','IO','NW','Memory']:
        v_remain.loc[pos_c,col]=v_remain.loc[pos_c,col] - df_applications.loc[pos_r,col]
      Res_instances[pos_r] =Res_instances[pos_r]-1
    B_star_res.loc[pos_r,pos_c]=-1
  return B_dash

In [ ]:
def getmatrix(A):
  num_machines = df_machines.shape[0]
  num_applications = df_applications.shape[0]
  B=[]
  sum=0
  for app in range(num_applications):
      b_temp=[]
      for mac in range(num_machines):
        b_temp.append(GLOBAL_ANS[app*num_machines + mac])
        sum+=GLOBAL_ANS[app*num_machines + mac]
      B.append(b_temp)
  return B,sum

In [ ]:
df_allocation_matrix = get_B_final(B_star)

pc = calculate_total_power_cost(df_allocation_matrix, df_applications, df_machines)
ac = calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix)
tc = calculate_total_system_cost(pc, ac)
asr = calculate_ASR(df_allocation_matrix)
acu = calculate_AU(df_allocation_matrix)

print(pc,ac,tc,asr,acu)